# Introduction to using stdpopsim

### Americas

**Date and Time:** Monday, February 22nd, 10-11:30AM PST; 1-2:30PM EST; 7-8:30PM CET; 5-6:30AM AEDT (Feb. 23rd)

**Instructor:** Andy Kern, University of Oregon

**Helpers:** Ilan Gronau, Elise Lauterbur, Manny Vasquez (slack channel: #feb22-23_americas)

### Transatlantic

**Date and Time:** Wednesday, February 24th, 5-6:30PM CET; 8-9:30AM PST; 11AM-12:30PM EST; 3-4:30AM AEDT (Feb. 25th)

**Instructor:** Jerome Kelleher, University of Oxford

**Helpers:** Ilan Gronau, Izabel Cavassim (slack channel: #feb24-25_transatlantic)

### Australasia

**Date and Time:** Friday, February 26th, 12-1:30PM AEDT; 2-3:30AM CET; 5-6:30PM PST (Feb. 25th); 8-9:30PM EST (Feb. 25th)

**Instructor:** Peter Ralph, University of Oregon

**Helpers:** Georgia Tsambos, Ariella Gladstein, Manny Vasquez (slack channel: #feb25-26_australasia)

## Workshop logistics

**Participants:** If you want, introduce yourselves in the Slack chat. What's your name? Where are you located? What do you work on?

**Banter/Questions**
Please use the Slack channel you were invited to for participant-participant help and chat. To ask a general question, simply write it in the channel, and to ask for one-one help DM one of the helpers in your session.

This worksheet was created by Ariella Gladstein, a postdoc at University of North Carolina, Chapel Hill.

## Workshop Outline
1. Basics of using Jupyter Notebooks
2. Overview of stdpopsim
3. How to use the Python API
4. How to use the command line interface
5. Example analysis
6. How to ask for help
7. Some examples of what stdpopsim cannot currently do
8. Teaser of how to contribute
9. Using stdpopsim after the workshop  
10. How to Navigate the stdpopsim library catalog

----------
## 1. Basics of using Jupyter Notebooks

Jupyter Notebooks have cells where you can write in Markdown and run code.  
To execute a cell, click the run button or press shift enter.

In [ ]:
print('Try writing some Python here')

In [ ]:
%%bash
echo 'We can also use Bash magic. Try writing some Bash here'

---------
## 2. Overview of stdpopsim

### What is stdpopsim?
- Library of previously published population genetic models that can be used to simulate data
- Includes simple & complex demographic models
- Models have undergone rigorous quality control to ensure what we implement matches the original publication

### Why is stdpopsim useful?
- Increase reproducibility in population genetics modeling
- Less work for simulating data to test new inference methods
- Facilitate comparisons among inference methods

### Phase 1
Adrion et al. (2020). _A community-maintained standard library of population genetic models_. eLife. https://doi.org/10.7554/eLife.54967

- Focused just on demographic modeling
- Uses msprime as simulation engine
- Realistic genetic maps for each species

### How to use stdpopsim
- Python API
  - more flexible
  - access to tskit functionality
  - faster for more replicates
- command line interface
  - one-liner
  - nice for people familiar with command line
  - easy to put into a bigger workflow

---------------
## 3. How to use the Python API

### Import the stdpopsim package

In [ ]:
import stdpopsim

### Open the catalog page

Resources:
- [Catalog](https://stdpopsim.readthedocs.io/en/stable/catalog.html#)
- [Documentation](https://stdpopsim.readthedocs.io/en/stable/api.html)
- [Tutorials](https://stdpopsim.readthedocs.io/en/stable/tutorial.html#running-stdpopsim-with-the-python-interface-api)

### Which species?

Let's simulate humans.

_Use the [catalog](https://stdpopsim.readthedocs.io/en/stable/catalog.html#) to find available species_

https://stdpopsim.readthedocs.io/en/stable/api.html#stdpopsim.get_species

In [ ]:
species = stdpopsim.get_species("HomSap")

### Which chromosome?

We'll simulate a region that is 10% of the length of chromosome 22.

Find what chromosomes are available in the [catalog](https://stdpopsim.readthedocs.io/en/stable/catalog.html#)

https://stdpopsim.readthedocs.io/en/stable/api.html#stdpopsim.Species.get_contig

In [ ]:
contig = species.get_contig("chr22", length_multiplier=0.1)

Using a *real* genetic map: (cannot be used in conjunction with `length_multiplier`)

In [ ]:
#contig = species.get_contig("chr22", genetic_map="HapMapII_GRCh37")

Using an alternative mutation or recombination rate:

In [ ]:
# other_contig = stdpopsim.Contig(
#     mutation_rate=contig.mutation_rate*0.5,
#     recombination_map=contig.recombination_map
# ) 



### Which demographic model?
https://stdpopsim.readthedocs.io/en/stable/api.html#stdpopsim.DemographicModel

Use the [catalog](https://stdpopsim.readthedocs.io/en/stable/catalog.html#) to find available demographic models.
You can also see them in the `species` object you just created:

In [ ]:
for model in species.demographic_models:
    print(model.id)

Then set your demographic model.
https://stdpopsim.readthedocs.io/en/stable/api.html#stdpopsim.Species.get_demographic_model

In [ ]:
model = species.get_demographic_model('OutOfAfrica_3G09')

You may want to verify that the simulated populations are correct.

In [ ]:
print([pop.id for pop in model.populations])

### How many samples?

In [ ]:
samples = model.get_samples(10, 10, 10)

### Which simulator?
https://stdpopsim.readthedocs.io/en/stable/api.html#simulation-engines

In [ ]:
engine = stdpopsim.get_engine('msprime')

We're now ready to simulate:
https://stdpopsim.readthedocs.io/en/stable/api.html#stdpopsim.Engine.simulate

In [ ]:
%%time
ts = engine.simulate(model, contig, samples, seed=1)

## Output
### to a tree sequence file

In [ ]:
ts.dump("OutOfAfrica_3G09_API.ts")

### to a vcf file

In [ ]:
with open("OutOfAfrica_3G09_API.vcf", "w") as vcf_file:
    ts.write_vcf(vcf_file, contig_id='22', ploidy=2)

Let's take a look at our vcf:

In [ ]:
%%bash
ls

In [ ]:
%%bash
head OutOfAfrica_3G09_API.vcf

### All together now!

In [ ]:
import stdpopsim
species = stdpopsim.get_species("HomSap")
model = species.get_demographic_model('OutOfAfrica_3G09')
samples = model.get_samples(10, 10, 10)
engine = stdpopsim.get_engine('msprime')
contig = species.get_contig("chr22", length_multiplier=0.1)
ts = engine.simulate(model, contig, samples)
ts.dump("OutOfAfrica_3G09_API.trees")
with open("OutOfAfrica_3G09_API.vcf", "w") as vcf_file:
    ts.write_vcf(vcf_file, contig_id='22', ploidy=2)

### _Try to simulate the Arabidopsis South Middle Atlas piecewise constant size model!_

In [ ]:
species = stdpopsim.get_species("") #fill me!
model = species.get_demographic_model('') #fill me!
samples = model.get_samples() #fill me!
engine = stdpopsim.get_engine('msprime')
contig = species.get_contig("", length_multiplier=0.1) #fill me!
ts = engine.simulate(model, contig, samples)

### Questions?

---------------
## 4. How to use the command line interface
Resources:
- [Documentation](https://stdpopsim.readthedocs.io/en/stable/cli_arguments.html)
- [Tutorials](https://stdpopsim.readthedocs.io/en/stable/tutorial.html#running-stdpopsim-with-the-command-line-interface-cli)

### Run stdpopsim with the help option

In [ ]:
%%bash
stdpopsim --help

`stdpopsim` uses a combination of [_positional arguments_](https://stdpopsim.readthedocs.io/en/stable/cli_arguments.html#Positional%20Arguments), which are required, and [_named arguments_](https://stdpopsim.readthedocs.io/en/stable/cli_arguments.html#Named%20Arguments), which are optional.

### Find your species and run stdpopsim with that species with the help option

In [ ]:
%%bash
stdpopsim HomSap --help

### Find your model and run stdpopsim with that model with the help option

In [ ]:
%%bash
stdpopsim HomSap \
        --help-models

## Run a simulation
- pick the number of samples for each population
- pick a chromosome
- do you want a fraction of the chromosome?
- decide on an output file

Let's do a dry run first by using `-D`.

In [ ]:
%%bash
stdpopsim HomSap 10 10 10 \
    --demographic-model OutOfAfrica_3G09 \
    --chromosome chr22 \
    --length-multiplier 0.1 \
    --output OutOfAfrica_3G09_CLI.ts \
    -D 

Does that look right? If so, let's do the simulation!

In [ ]:
%%bash
stdpopsim HomSap 10 10 10 \
         --demographic-model OutOfAfrica_3G09 \
         --chromosome chr22 \
         --length-multiplier 0.1 \
         --output OutOfAfrica_3G09_CLI.ts

In [ ]:
%%bash
tskit vcf --ploidy 2 OutOfAfrica_3G09_CLI.ts > OutOfAfrica_3G09_CLI.vcf

Let's take a look at our vcf.

In [ ]:
%%bash
ls

In [ ]:
%%bash
head OutOfAfrica_3G09_CLI.vcf

### _Try to simulate the Arabidopsis South Middle Atlas piecewise constant size model!_

Bonus: from the `--help` output, can you see how you might use one of the published genetic maps in this simulation? (perhaps the *SalomeAveraged_TAIR7* map?)

In [ ]:
%%bash
stdpopsim #fill me!

### Questions?

-------------
## 5. Example analysis
Let's suppose we wanted to check if a published model is a good approximation of our real data. To do this, we'll calculate a few population genetics statistics and see if the real data overlap our simulated data. (For this exercise we will assume some value for the stats from the real data).


### Our pretend real data
Let's say we have samples from three human populations, and they have a mean genetic diversity of: 0.00045, 0.00029, 0.00027.

In [ ]:
piA = 0.00045
piB = 0.00029
piC = 0.00027

### Simulate a model to compare to real data and calculate stats

We'll simulate the human Three population out-of-Africa _n_ times.  

_Find this model in the catalog._  
_How many populations are there?_  
_How many samples do we want to simulate?_  
_What chromosome do we want to simulate?_  
_Do we want to reduce the chromosome size?_

#### Set up the simulation

In [ ]:
import stdpopsim
species = stdpopsim.get_species("HomSap")
model = species.get_demographic_model('OutOfAfrica_3G09')
samples = model.get_samples(10, 10, 10)
engine = stdpopsim.get_engine('msprime')
contig = species.get_contig("chr22", length_multiplier=0.1)

#### Simulation and calculate the stats in a for loop

Make a list of sample chromosomes (nodes) from each population

In [ ]:
def make_sample_list(ts):
    sample_list = []
    for pop in range(0, ts.num_populations):
        sample_list.append(ts.samples(pop).tolist())
    return sample_list

Run the simulations and calculate the summary statistics

In [ ]:
%%time
from progressbar import ProgressBar #This is just to get a progress bar
pbar = ProgressBar() #This is just to get a progress bar

n = 50
pi_list = []
for i in pbar(range(n)):
    ts = engine.simulate(model, contig, samples)
    sample_list = make_sample_list(ts)
    pi_list.append(ts.diversity(sample_sets=sample_list))
print('Done simulating {} replicates!'.format(n))

### Plot stats from simulated and "real" data

Convert simulated stats to a dataframe (only dataframes are easy to work with), then lets look at the first couple of lines using `head()`

In [ ]:
import pandas as pd
pi_df = pd.DataFrame(
    data=pi_list, columns=[pop.id for pop in model.populations])
pi_df.head()

In [ ]:
pi_df_melted = pd.melt(pi_df, var_name='Pops', value_name='Pi')

## Plot data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
ax = sns.histplot(data=pi_df_melted, x="Pi", hue="Pops", bins=30)
ax.axvline(x=piA, color='blue')
ax.axvline(x=piB, color='orange')
ax.axvline(x=piC, color='green')

## A PCA example using scikit-allel
Quick look at using stdpopsim together with scikit-allel to examine PCA of genetic variation from simulations.

First we set the model:

In [ ]:
import allel
import stdpopsim

species = stdpopsim.get_species("HomSap")
model = species.get_demographic_model('OutOfAfrica_3G09')
samples = model.get_samples(50, 50, 50)
engine = stdpopsim.get_engine('msprime')
contig = species.get_contig("chr22", length_multiplier=0.1)

next we will simulate and use tools from scikit-allel to go from tree sequences to genotypes. we'll output the first few rows of the genotype array produced by allel

In [ ]:
ts = engine.simulate(model, contig, samples)
haps = allel.HaplotypeArray(ts.genotype_matrix())
gns = haps.to_genotypes(ploidy=2)
gns

now we're ready to do PCA. we'll do the decomposition and then plot, note that allel requires a flattened version of the genotype array above that codes individual genotypes as 0/1/2 according to the number of reference alleles

In [ ]:

def plot_pca_coords(coords, fitted, pc1, pc2, ax):
    x = coords[:, pc1]
    y = coords[:, pc2]
    ax.plot(x[0:25], y[0:25], marker='o', linestyle=' ', label='YRI', markersize=6)
    ax.plot(x[25:50], y[25:50], marker='o', linestyle=' ', label='CEU', markersize=6)
    ax.plot(x[50:75], y[50:75], marker='o', linestyle=' ', label='CHB', markersize=6)

    ax.set_xlabel('PC%s (%.1f%%)' % (pc1+1, fitted.explained_variance_ratio_[pc1]*100))
    ax.set_ylabel('PC%s (%.1f%%)' % (pc2+1, fitted.explained_variance_ratio_[pc2]*100))


fig, ax = plt.subplots(figsize=(6, 6))
sns.despine(ax=ax, offset=10)

coords, fitted = allel.pca(gns.to_n_ref())
plot_pca_coords(coords, fitted, 0, 1, ax)
ax.legend()



### Questions?

---------
## 6. How to ask for help
- Have you read the [documentation](https://stdpopsim.readthedocs.io/en/stable/index.html)?
- Search open and closed [GitHub issues](https://github.com/popsim-consortium/stdpopsim/issues?q=is%3Aissue)
- Write a new [GitHub issue](https://github.com/popsim-consortium/stdpopsim/issues/new/choose)
- Join the PopSim Slack workspace and post in the #newbie-help channel (An invitation has been sent to all participants. If you have not recieved an invitation, email Ariella, Andy, or Peter)

---------
## 7. Some examples of what stdpopsim cannot currently do
- simulate species or demographic models that are not in the catalog 
    - if you want to do this, if it is a published model - submit it to stdpopsim, if it is not a published model, use a simulator (e.g. msprime, slim)
- simulate parameter values not from the published model (including priors)
- simulate selection (in the works!)
- simulate missing data and errors (on the horizon!)


----------
## 8. Teaser of [how to contribute](https://stdpopsim.readthedocs.io/en/stable/development.html#)

- Write [GitHub issues](https://github.com/popsim-consortium/stdpopsim/issues/new/choose)!
- [Add new species](https://stdpopsim.readthedocs.io/en/stable/development.html#adding-a-new-species)
- [Add new demographic models](https://stdpopsim.readthedocs.io/en/stable/development.html#adding-a-new-demographic-model)
- Help with Documentation and tutorials

-----------
## 9. Using stdpopsim on your own after the workshop
- Play in a Jupyter Notebook Binder
  - Using this one [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/popsim-consortium/workshops.git/main?filepath=intro_stdpopsim%2FIntro_stdpopsim.ipynb)
  - In the Binder associated with the [stdpopsim GitHub repository](https://github.com/popsim-consortium/stdpopsim) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/popsim-consortium/stdpopsim/master?filepath=stdpopsim_example.ipynb)
- Install stdpopsim locally following the [instructions in the documentation](https://stdpopsim.readthedocs.io/en/stable/installation.html)
- Consult the [stdpopsim documentation](https://stdpopsim.readthedocs.io/en/stable/installation.html)

---------
## 10. How to Navigate the [stdpopsim library catalog](https://stdpopsim.readthedocs.io/en/stable/catalog.html)

### The Catalog is organized first by species.

_How many species are there?_

![](images/catalog.png)


### Each species has a set of defining attributes. 

_What are the attributes?_

![](images/species_attributes.png)

### Each species has defined genome parameters.

_What are the genome parameters?_

![](images/genome_params.png)

### Some species have a genetic map.

Genetic maps are stored on AWS and downloaded to cache when used.

![](images/genetic_maps.png)

### Some species have demographic models.

All models are from published models.

_What models are available? Are there any you recognize from the literature?_

![](images/models.png)

### Each model has a description and set of defining attributes. 

_What are the attributes?_

![](images/model_attr.png)

### Each model has a table of defined model parameters from the publication. 

_Can you find where in the original publication the model parameters are given?_

![](images/model_params.png)